<a href="https://colab.research.google.com/github/parthxtripathi/LPP-Lottery-Problem/blob/main/LPP_Lottery_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing package
!pip install cvxopt
!pip install cvxpy
!pip install mip
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 105.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 114.3 MB/s eta 0:00:00


In [ ]:
#import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import notebook
import numpy as np
import cvxpy as cp
from itertools import combinations 
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
import cvxopt
import cvxpy
from mip import Model, xsum, maximize, minimize, BINARY
import pulp as plp
import random
import copy

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 31.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
#For referecne, display available solvers for CVXPY
print(cvxpy.installed_solvers())

(CVXPY) Apr 20 07:29:06 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Apr 20 07:29:06 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


In [ ]:
# generate alll tickets and possible draws given the problem parameters
def generate(n,k,p,t):
    comb_tickets = combinations([i+1 for i in range(n)], k) 
    tickets = np.array([set(x) for x in list(comb_tickets)])

    comb_draws = combinations([i+1 for i in range(n)], p) 
    draws = np.array([set(x) for x in list(comb_draws)])
    
    coef_matrix = np.zeros((len(draws), len(tickets)), dtype=int)
    
    for i in notebook.tqdm(range(coef_matrix.shape[0])):
        for j in range(coef_matrix.shape[1]):
            intersection = len(draws[i] & tickets[j])
            coef_matrix[i,j] = 1 if intersection >= t else 0

    return draws, tickets, coef_matrix

In [ ]:
def MIP_Optmise(coef_matrix):
    
    w = coef_matrix
    T = range(w.shape[1])
    D = range(w.shape[0])
    m = Model('lotto')

    x = [m.add_var(var_type=BINARY) for i in T]

    m.objective = minimize(xsum(x[i] for i in T))

    for i in D:
        m += (xsum(w[i][j]* x[j] for j in T) >= 1)

    #print(m.optimize(max_seconds=60))
    print(m.optimize())
    
    selected = [i for i in T if x[i].x >= 0.99]
    #print('selected items: {}'.format(selected))
    
    deselected = [i for i in T if x[i].x <= 0.99]
    
    return selected, deselected  

In [ ]:
def CVXPY_Optmise(coef_matrix):
    
    M = coef_matrix

    selection = cp.Variable(M.shape[1], boolean = True)
    ones_vec = np.ones(M.shape[1], dtype=int)
    
    constraints = []
    
    for i in range(M.shape[0]):
        constraints.append(M[i] * selection >= 1)

    cost = ones_vec * selection

    problem = cp.Problem(cp.Minimize(cost), constraints)

    print(problem.solve(solver=cp.GLPK_MI))

    selected = np.nonzero(selection.value > 0.75)
    #print('selected items: {}'.format(list(selected[0])))
    
    deselected = np.nonzero(selection.value < 0.75)
    
    return list(selected[0]), list(deselected[0])  

In [ ]:
def OR_Optmise(coef_matrix):    
    w = coef_matrix
    T = range(w.shape[1])
    D = range(w.shape[0])
    
    # Create the mip solver with the CBC backend.
    solver = pywraplp.Solver('simple_mip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # Variables
    x = {}
    for i in T:
        x[i] = solver.BoolVar('x_%i' % (i))
    
    # The amount packed
    for i in D:
        solver.Add(solver.Sum(w[i][j] * x[j] for j in T) >= 1)  
        
    solver.Minimize(solver.Sum([x[j] for j in T]))
      
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        print('An optimal feasible solution was found.')
    elif status == pywraplp.Solver.FEASIBLE:
        print('A feasible solution was found, but we dont know if its optimal.')
    elif status == pywraplp.Solver.INFEASIBLE:
        print('The problem was proven infeasible.')
        return
    elif status == pywraplp.Solver.MODEL_INVALID:
        print('The given CpModelProto didnt pass the validation step.')
        return
    else:
        print('The status of the model is unknown because a search limit was reached.')
        return
         
    # The objective value of the solution.
    print('WallTime = %f mSecs' % solver.WallTime())
    
    selected = [i for i in T if x[i].solution_value() == 1]
    #print('selected items: {}'.format(selected))
    
    deselected = [i for i in T if x[i].solution_value() < 1]
    
    return selected, deselected  

In [ ]:
def OR_ContraintProgramming(coef_matrix, target, greedy_initialisation = False):    
    
    w = coef_matrix
    T = range(w.shape[1])
    D = range(w.shape[0])
    
    # Create the mip solver with the CBC backend.
    model = cp_model.CpModel()
    
    # Variables
    x = {}
    for i in T:
        x[i] = model.NewBoolVar('x_%i' % (i))
    
    if greedy_initialisation == True:
        print("Greedy Init")
        selected, deselected = Greedy_Optmise(coef_matrix)
        for i in selected:
            model.AddHint(x[i], True)
        
    #break symmetry when it exists to speed up solution finding
    model.Add(x[0] == 1) 
    
    model.Add(sum([x[j] for j in T])==target)
    
    # The amount packed
    for i in D:
        model.Add(sum(w[i][j] * x[j] for j in T) >= 1)  
              
    solver = cp_model.CpSolver()

    status = solver.Solve(model)
    
    print(solver.ResponseStats())
    
    if status == cp_model.OPTIMAL:
        print('An optimal feasible solution was found.')
    elif status == cp_model.FEASIBLE:
        print('A feasible solution was found, but we dont know if its optimal.')
    elif status == cp_model.INFEASIBLE:
        print('The problem was proven infeasible.')
        return [], []
    elif status == cp_model.MODEL_INVALID:
        print('The given CpModelProto didnt pass the validation step.')
        return [], []
    else:
        print('The status of the model is unknown because a search limit was reached.')
        return [], []
    
    selected = [i for i in T if solver.Value(x[i]) == 1]
    print('selected items: {}'.format(selected))
    
    deselected = [i for i in T if solver.Value(x[i]) < 1]
    
    return selected, deselected  

In [ ]:
def PULP_Optmise(coef_matrix):    
    w = coef_matrix
    T = range(w.shape[1])
    D = range(w.shape[0])
       
    opt_model = plp.LpProblem(name="MIP_Model")
   
    # if x is Binary
    x_vars  = {(i): plp.LpVariable(cat=plp.LpBinary, name="x_{0}".format(i)) for i in T}

    # >= constraints
    constraints = {i : opt_model.addConstraint(plp.LpConstraint(e=plp.lpSum(w[i,j] * x_vars[j] for j in T),
                         sense=plp.LpConstraintGE,
                         rhs=1,
                         name="constraint_{0}".format(i))) for i in D}

    objective = plp.lpSum(x_vars[i] for i in T)

    # for minimization
    opt_model.sense = plp.LpMinimize
    opt_model.setObjective(objective)

    # solving with CBC
    opt_model.solve()
    
    selected = [i for i in T if x_vars[i].varValue == 1]
    #print('selected items: {}'.format(selected))
    
    deselected = [i for i in T if x_vars[i].varValue < 1]
    
    return selected, deselected  

In [ ]:
def RemoveRedundant(coef_matrix, selected, deselected):
    
    coef_matrix_red = np.zeros(coef_matrix.shape, dtype=int)
    
    for i in selected:
        coef_matrix_red[:,i] = coef_matrix[:,i]
        
    v = np.count_nonzero(coef_matrix_red, axis=1)
    
    selected_copy = copy.deepcopy(selected)
    for i in selected_copy:
        if(min(v[np.nonzero(coef_matrix_red[:,i])])>1):
            selected.remove(i)
            deselected.append(i)
            v = np.subtract(v, coef_matrix_red[:,i])
            
    return selected, deselected

In [ ]:
def Greedy_Optmise(coef_matrix):
    # let solution be empty
    X = set()
    # for each ticket compute how many draws it covers
    d = np.count_nonzero(coef_matrix, axis=0)
    # initialise set of uncovered draws i.e. all uncovered
    I = set(range(coef_matrix.shape[0]))
    # initialise set of unselected tickets i.e. none selected
    J = set(range(coef_matrix.shape[1]))
    # select first ticket
    w = 0
    # add to the solution
    X.add(w)
    # remove the ticket from the unselected list
    J.remove(w)
    # get draws it covers
    Dw = set(np.nonzero(coef_matrix[:,w])[0])
    # remove covered draws from the uncovered set
    I = I - Dw
    for i in Dw:
        d = np.subtract(d, coef_matrix[i,:])
    while I != set():
        print("-", end="")
        # select random ticket that maximizes uncovered draws
        f = (np.where(d == np.amax(d))[0]) 
        w = random.choice(f)
        # add to the solution
        X.add(w)
        # remove the ticket from the unselected list
        J.remove(w)
        # get draws it covers
        Dw = set(np.nonzero(coef_matrix[:,w])[0])
        Dw = Dw.intersection(I)
        # remove covered draws from the uncovered set
        I = I - Dw 
        for i in Dw:
            d = np.subtract(d, coef_matrix[i,:]) 
    print("")
    
    return list(X), list(J)

In [ ]:
def Improve(coef_matrix, selected, deselected, factor=0.5, algo='or'):
    #select % of solution
    sampling = random.sample(selected, k=int(len(selected)*factor))
    print(" Keeping ", len(sampling), "tickets ouf of" , len(selected))
    reduced_coef_matrix = coef_matrix[:,sampling]
    #check which draws are no longer covered
    v = np.count_nonzero(reduced_coef_matrix, axis=1)
    not_coverd_draws = np.where(v == 0)[0]
    
    others_tickets = list(set(range(coef_matrix.shape[1]))-set(sampling))
    
    reduced_coef_matrix = coef_matrix[:, others_tickets]
    reduced_coef_matrix = reduced_coef_matrix[not_coverd_draws, :]
    
    print(" OR Size = ", reduced_coef_matrix.shape)
    
    
    if algo == 'cvx':
        print('CVXPY Optimise')
        x, y = CVXPY_Optmise(reduced_coef_matrix)
        
    if algo == 'mip':
        print('MIP Optimise')
        x, y = MIP_Optmise(reduced_coef_matrix) 
        
    if algo == 'or':
        print('OR Optimise')
        x, y = OR_Optmise(reduced_coef_matrix)  

    if algo == 'pulp':
        print('PULP Optimise')
        x, y = PULP_Optmise(reduced_coef_matrix)      
    
    
    for i in x:
        sampling.append(others_tickets[i])
       
    others_tickets = list(set(range(coef_matrix.shape[1]))-set(sampling))
        
    return RemoveRedundant(coef_matrix, sampling, others_tickets)

In [ ]:
def Greedy_Optmise_Tries(coef_matrix, tries=1, factor=0.5, algo = 'or'):
    selected, deselected = Greedy_Optmise(coef_matrix)
    print("Original Solution Tickets =", len(selected))

    selected, deselect = RemoveRedundant(coef_matrix, selected, deselected)
    print("Remove Redundant Tickets =", len(selected))

    for i in range(tries):
        print("Iteration = ", i, end="")
        selected, deselect = Improve(coef_matrix, selected, deselected, factor, algo)
        print("Improved Tickets =", len(selected))

    return selected, deselected    

In [ ]:
#get tickets that provides the solution to our problem
def get_tickets(draws, tickets, coef_matrix, deselected):
    tickets_str = [str(x) for x in tickets]
    draws_str = [str(x) for x in draws]

    coefficient_matrix = pd.DataFrame(coef_matrix, index = draws_str, columns = tickets_str)
    coefficient_matrix = coefficient_matrix.drop(coefficient_matrix.columns[deselected], axis=1)
    
    return coefficient_matrix

In [ ]:
# perform the optmisation using the library of our choice
def get_reduced_wheel(n,k,p,t,algo, target=0, greedy_initialisation = False, factor=0.5, tries=1):
    
    draws, tickets, coef_matrix = generate(n,k,p,t)

    number_tickets = len(tickets)
    print('Tickets =', number_tickets)

    number_draws = len(draws)
    print('Draws =', number_draws)
    
    selected, deselected =  Greedy_Optmise_Tries(coef_matrix, tries, factor, algo)
        
    optimal_tickets = get_tickets(draws, tickets, coef_matrix, deselected)
    
    return optimal_tickets

In [ ]:
n = 12  # numbers in the set
k = 5  # numbers in ticket
p = 4  # numbers in draw
t = 3 # at least match

In [ ]:
%%time
Greedy_tickets = get_reduced_wheel(n,k,p,t,algo = 'cvx', factor=0.75, tries=10)

  0%|          | 0/495 [00:00<?, ?it/s]

Tickets = 792
Draws = 495
-------------
Original Solution Tickets = 14
Remove Redundant Tickets = 14
Iteration =  0 Keeping  10 tickets ouf of 14
 OR Size =  (62, 782)
CVXPY Optimise


/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9/dis

4.0
Improved Tickets = 14
Iteration =  1 Keeping  10 tickets ouf of 14
 OR Size =  (41, 782)
CVXPY Optimise
4.0
Improved Tickets = 14
Iteration =  2 Keeping  10 tickets ouf of 14
 OR Size =  (57, 782)
CVXPY Optimise


/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 64 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 65 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9/d

...

4.0
Improved Tickets = 14
Iteration =  8 Keeping  10 tickets ouf of 14
 OR Size =  (53, 782)
CVXPY Optimise


/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 443 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 444 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9

4.0
Improved Tickets = 14
Iteration =  9 Keeping  10 tickets ouf of 14
 OR Size =  (52, 782)
CVXPY Optimise


/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 497 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 498 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.9

4.0
Improved Tickets = 14
CPU times: user 2.66 s, sys: 58.5 ms, total: 2.72 s
Wall time: 3.07 s


In [ ]:
Greedy_tickets

,"{1, 2, 3, 4, 5}","{1, 2, 6, 8, 11}","{1, 2, 9, 10, 11}","{1, 3, 6, 9, 12}","{1, 4, 6, 9, 12}","{1, 5, 7, 10, 11}","{1, 7, 8, 10, 12}","{2, 3, 4, 8, 10}","{2, 3, 5, 7, 8}","{2, 4, 6, 7, 9}","{3, 4, 7, 11, 12}","{3, 6, 9, 10, 11}","{4, 5, 6, 10, 12}","{5, 8, 9, 11, 12}"
"{1, 2, 3, 4}",1,0,0,0,0,0,0,1,0,0,0,0,0,0
"{1, 2, 3, 5}",1,0,0,0,0,0,0,0,1,0,0,0,0,0
"{1, 2, 3, 6}",1,1,0,1,0,0,0,0,0,0,0,0,0,0
"{1, 2, 3, 7}",1,0,0,0,0,0,0,0,1,0,0,0,0,0
"{8, 1, 2, 3}",1,1,0,0,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"{8, 9, 10, 11}",0,0,1,0,0,0,0,0,0,0,0,1,0,1
"{8, 9, 10, 12}",0,0,0,0,0,0,1,0,0,0,0,0,0,1
"{8, 9, 11, 12}",0,0,0,0,0,0,0,0,0,0,0,0,0,1
"{8, 10, 11, 12}",0,0,0,0,0,0,1,0,0,0,0,0,0,1
